在 Python 中，数据的属性和处理数据的方法统称为属性（attribute），方法只是可调用的属性。

我们可以创建特性（property），在不改变接口的前提下，使用存取方法（读值方法、设值方法）修改数据属性。

统一访问原则：不管服务是由存储还是计算实现的，一个模块提供的所有服务都应该通过统一的方式使用。

使用点号访问属性时（如 obj.attr），Python 解释器会调用特殊的方法（如 \_\_getattr__ 和 \_\_setattr__）计算属性。

动态创建属性是一种元编程，框架的作者经常这么做。

In [1]:
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'osconfeed.json'


def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)  # 下载
        with urlopen(URL) as remote, open(JSON, 'wb') as local:  # 使用两个上下文管理器
            local.write(remote.read())

    with open(JSON) as fp:
        return json.load(fp)  # 解析 json 文件，返回 Python 原生对象

osconfeed.json 中每个元素都有一个名为 serial 的字段，这是元素在各个列表中的唯一标识符。

In [2]:
# 字典，嵌套字典和列表
feed = load()

# 列出 Schedule 键中的 4 个记录集合
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [3]:
for key, value in sorted(feed['Schedule'].items()):
    print('{:3} {}'.format(len(value), key))  # 显示各个集合中的记录数量

  1 conferences
484 events
357 speakers
 53 venues


In [4]:
feed['Schedule']['speakers'][-1]['name']  # 最后一个演讲者的名字

'Carina C. Zona'

In [5]:
feed['Schedule']['speakers'][-1]['serial']  # 最后一个演讲者的编号

141590

In [6]:
feed['Schedule']['events'][0]['name']

'Migrating to the Web Using Dart and Polymer - A Guide for Legacy OOP Developers'

In [7]:
feed['Schedule']['events'][0]['speakers']  # 事件的演讲者

[149868]

在 Python 中，可以实现一个近似字典的类。

仅当无法使用常规的方式获取属性（即在实例、类或超类中抄不到指定的属性），解释器才会调用特殊的 \_\_getattr__ 方法。

In [8]:
from collections import abc


class FrozenJSON:
    """A read-only façade for navigating a JSON-like object
       using attribute notation
    """

    def __init__(self, mapping):
        self.__data = dict(mapping)  # 构建字典

    def __getattr__(self, name):  # 仅当没有指定名称的属性时才会调用
        if hasattr(self.__data, name):
            return getattr(self.__data, name)  # 返回属性
        else:
            return FrozenJSON.build(self.__data[name])  # 返回对应元素

    @classmethod
    def build(cls, obj):  # 备选构造方法
        if isinstance(obj, abc.Mapping):  # 如果是映射，构建 FrozenJSON 对象
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):  # 如果是列表
            return [cls.build(item) for item in obj]
        else:  # 否则直接返回
            return obj

In [9]:
raw_feed = load()
feed = FrozenJSON(raw_feed)  # 传入嵌套的字典和列表
len(feed.Schedule.speakers)  # 遍历字典，获取长度

357

In [10]:
sorted(feed.Schedule.keys())  # 获取记录集合的名称

['conferences', 'events', 'speakers', 'venues']

In [11]:
for key, value in sorted(feed.Schedule.items()): # 获取各个记录集合及其内容
    print('{:3} {}'.format(len(value), key))

  1 conferences
484 events
357 speakers
 53 venues


In [12]:
feed.Schedule.speakers[-1].name  # 映射元素转换为 FrozenJSON 对象，列表仍然是列表

'Carina C. Zona'

In [13]:
talk = feed.Schedule.events[0]
type(talk)  # FrozenJSON 实例

__main__.FrozenJSON

In [14]:
talk.name

'Migrating to the Web Using Dart and Polymer - A Guide for Legacy OOP Developers'

In [15]:
talk.speakers  # 列出演讲者的编号

[149868]

In [16]:
talk.flavor  # 抛出 KeyError 异常

KeyError: 'flavor'

从随机源中生成或 仿效动态属性名的脚本都必须处理一个问题：原始数据中的键可能不适合作为属性名。

keyword 模块的 iskeyword 函数可以检查字符串是否为关键字。

str 类提供的 s.isidentifier() 方法能根据语言的语法判断 s 是否为有效的 Python 标识符。

从数据中创建实例属性的名称时肯定有可能会引入缺陷，因为类属性（例如方法）可能被覆盖，或者由于以外覆盖现有的实例属性而丢失数据。

In [17]:
grad = FrozenJSON({'name': 'Jim Bo', 'class': 1982})

In [18]:
grad.class

SyntaxError: invalid syntax (<ipython-input-18-bb5c99ef29c5>, line 1)

In [19]:
getattr(grad, 'class')

1982

In [20]:
from collections import abc
import keyword


class FrozenJSON:
    """A read-only façade for navigating a JSON-like object
       using attribute notation
    """

    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):  # 检查是否为标识符
                key += '_'
            self.__data[key] = value

    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON.build(self.__data[name])

    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:  # <8>
            return obj

In [21]:
grad = FrozenJSON({'name': 'Jim Bo', 'class': 1982})
grad.name

'Jim Bo'

In [22]:
grad.class_

1982

用于构建实例的是特殊方法 \_\_new__：这是个类方法（使用特殊方式处理，因此不必使用 @classmethod 装饰器），必须返回一个实例。返回的实例会作为第一个参数（即 self）传给 \_\_init__ 方法。

所以可以说 \_\_new__ 是构造方法，而 \_\_init__ 是初始化方法。

\_\_new__ 的第一个参数是类本身（因为创建的对象通常是那个类的实例），余下的参数与 \_\_init__ 方法一样，只不过没有 self 。

In [23]:
from collections import abc
import keyword


class FrozenJSON:
    """A read-only façade for navigating a JSON-like object
       using attribute notation
    """

    def __new__(cls, arg):  # 类方法
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls)  # 默认行为是委托给超类的 __new__ 方法
        elif isinstance(arg, abc.MutableSequence):
            return [cls(item) for item in arg]
        else:
            return arg

    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):
                key += '_'
            self.__data[key] = value

    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON(self.__data[name])  # 只需调用构造犯法

In [24]:
raw_feed = load()
feed = FrozenJSON(raw_feed)
len(feed.Schedule.speakers)

357

In [25]:
sorted(feed.Schedule.keys())

['conferences', 'events', 'speakers', 'venues']

In [26]:
feed.Schedule.speakers[-1].name

'Carina C. Zona'

In [27]:
talk = feed.Schedule.events[0]
talk.name

'Migrating to the Web Using Dart and Polymer - A Guide for Legacy OOP Developers'

In [28]:
talk.speakers

[149868]

In [29]:
talk.flavor

KeyError: 'flavor'

pickle（泡菜）是 Python 对象序列化格式的名字，还是在那个格式与对象之间相互转换的某个模块的名字。

标准库中有个 shelve（架子）模块，shelve.open 高阶函数返回一个 shelve.Shelf 实例，这是简单的键值对象数据库，背后由 dbm 模块支持，具有以下特点
- shelve.Shelf 是 abc.MutableMapping 的子类，因此提供了处理映射类型的重要方法
- shelve.Shelf 还提供了几个管理 I/O 的方法，如 sync 和 close ；它也是一个上下文管理器
- 只要把新值赋予键，就会保存键和值
- 键必须是字符串
- 值必须是 pickle 模块能处理的对象

In [30]:
import warnings

DB_NAME = 'schedule1_db'
CONFERENCE = 'conference.115'


class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)  # 使用关键字参数传入的属性构建实例


def load_db(db):
    raw_data = load()  # 加载数据
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():  # 迭代集合
        record_type = collection[:-1]  # 去掉尾部的 s
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])  # 组成键
            record['serial'] = key  # serial 设置为键
            db[key] = Record(**record)  # 构建 Record 实例

In [31]:
import shelve

db = shelve.open(DB_NAME)  # 打开现有的数据库文件，或新建一个

if CONFERENCE not in db:  # 检查键是否存在，也可以使用 len(db) 判断，对大型 dbm 数据库就很费时
    load_db(db)  # 加载数据

speaker = db['speaker.157509']  # 获取记录
type(speaker)  # Record 实例

<ipython-input-30-9041ce2d2869>:14: UserWarning: loading schedule1_db
  warnings.warn('loading ' + DB_NAME)


__main__.Record

In [32]:
speaker.name, speaker.twitter

('Robert Lefkowitz', 'sharewaveteam')

In [33]:
db.close()  # 关闭 shelve.Shelf 对象

对象的 \_\_dict__ 属性中存储着对象的属性，前提是类中没有声明 \_\_slots__ 属性。

自定义的异常通常是标志类，没有定义体。写一个文档字符串，说明异常的用途。

特性（property）是用于管理实例属性的类属性。特性经常用于把公开的属性变成使用读值方法和设值方法管理的属性，且在不影响客户端代码的前提下实施业务规则。

In [34]:
import warnings
import inspect

DB_NAME = 'schedule2_db'  # 数据库文件
CONFERENCE = 'conference.115'


class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __eq__(self, other):  # <3>
        if isinstance(other, Record):
            return self.__dict__ == other.__dict__
        else:
            return NotImplemented

# 需要数据库但没有指定数据库时抛出
class MissingDatabaseError(RuntimeError):
    """Raised when a database is required but was not set."""


# 扩展 Record 类，添加对数据库的支持：为了操作数据库，必须为 DbRecord 提供一个数据库引用
class DbRecord(Record):  # <2>

    __db = None  # 属性

    @staticmethod  # 静态方法，设置 __db 属性
    def set_db(db):
        DbRecord.__db = db  # <5>

    @staticmethod  # 静态方法，获取 __db 属性
    def get_db():
        return DbRecord.__db

    @classmethod  # 类方法，从数据库中获取记录
    def fetch(cls, ident):
        db = cls.get_db()
        try:
            return db[ident]  # 获取键对应的记录
        except TypeError:
            if db is None:  # 抛出自定义异常
                msg = "database not set; call '{}.set_db(my_db)'"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else:  # 重新抛出异常
                raise

    def __repr__(self):  # 辅助调试和测试
        if hasattr(self, 'serial'):  # 如果有 serial 属性，在字符串表示形式中使用
            cls_name = self.__class__.__name__
            return '<{} serial={!r}>'.format(cls_name, self.serial)
        else:
            return super().__repr__()  # 调用继承的 __repr__ 方法


# 扩展 DbRecord 类
class Event(DbRecord):

    @property  # 特性
    def venue(self):
        key = 'venue.{}'.format(self.venue_serial)
        return self.__class__.fetch(key)  # 调用 DnRecord 类的 fetch 类方法

    @property
    def speakers(self):
        if not hasattr(self, '_speaker_objs'):  # 检查是否有 _speaker_objs 属性
            spkr_serials = self.__dict__['speakers']  # 没有，从 __dict__ 实例属性中获取
            fetch = self.__class__.fetch  # 获取 fetch 类方法的引用
            self._speaker_objs = [fetch('speaker.{}'.format(key))
                                  for key in spkr_serials]  # 获取 speaker 记录列表
        return self._speaker_objs  # 返回记录列表

    def __repr__(self):  # 辅助调试和测试
        if hasattr(self, 'name'):  # 如果已有 name 属性，在字符串表示形式中使用
            cls_name = self.__class__.__name__
            return '<{} {!r}>'.format(cls_name, self.name)
        else:
            return super().__repr__()  # 调用继承的 __repr__ 方法

        
def load_db(db):
    raw_data = load()
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1]  # 去掉尾部的 s
        cls_name = record_type.capitalize()  # 首字母大写
        cls = globals().get(cls_name, DbRecord)  # 从全局作用域获取名称对应的对象
        if inspect.isclass(cls) and issubclass(cls, DbRecord):  # 类，DbRecord 的子类
            factory = cls  # 把对象赋值给 factory 变量
        else:
            factory = DbRecord  # 把 DbRecord 赋值给 factory 变量
        for record in rec_list:  # 创建 key 保存记录
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            db[key] = factory(**record)  # 由 factory 构建存储对象

In [35]:
db = shelve.open(DB_NAME)
load_db(db)
DbRecord.set_db(db)  
event = DbRecord.fetch('event.33950')  # 能获取任何类型的记录
event  # Event 类的实例

<ipython-input-34-69e0de1ffef8>:83: UserWarning: loading schedule2_db
  warnings.warn('loading ' + DB_NAME)


<Event 'There *Will* Be Bugs'>

In [36]:
event.venue  # 返回 DbRecord 实例

<DbRecord serial='venue.1449'>

In [37]:
event.venue.name

'Portland 251'

In [38]:
for spkr in event.speakers:  # 迭代
    print('{0.serial}: {0.name}'.format(spkr))

speaker.3471: Anna Martelli Ravenscroft
speaker.5199: Alex Martelli


In [39]:
db.close()

假设有个销售散装有机食物的电商应用，客户可以按重量订购见过、干果或杂粮。下面定义了商品类：

In [40]:
class LineItem:

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [41]:
raisins = LineItem('Golden raisins', 10, 6.95)
raisins.weight, raisins.description, raisins.price

(10, 'Golden raisins', 6.95)

In [42]:
raisins.weight = -20  # 无效输入
raisins.subtotal()    # 无效输出

-139.0

抽象特性的定义有两种方式
- 使用特性工厂函数
- 使用描述符类

特性本身就是使用描述符类实现的

In [43]:
# 使用装饰器定义特性
class LineItem:

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight  # 使用特性的设值方法
        self.price = price

    def subtotal(self):
        return self.weight * self.price

    @property  # 装饰读值方法
    def weight(self):  # 实现特性的方法，其名称都与公开属性的名称一样
        return self.__weight  # 真正的值存储在私有属性 __weight 中

    @weight.setter  # 绑定设值方法
    def weight(self, value):
        if value > 0:
            self.__weight = value  # 大于 0
        else:
            raise ValueError('value must be > 0')  # 小于 0 ，抛出异常

In [44]:
walnuts = LineItem('walnuts', 0, 10.00)

ValueError: value must be > 0

内置的 property 经常用作装饰器，但它其实是一个类。

property 构造方法的完整签名：property(fget=None, fset=None, fdel=None, doc=None) 所有参数都是可选的。

In [45]:
# 不使用装饰器定义特性
class LineItem:

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

    def get_weight(self):  # 读值方法
        return self.__weight

    def set_weight(self, value):  # 设值方法
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')

    weight = property(get_weight, set_weight)  # 构建 property 对象，赋值给公开的类属性

In [46]:
walnuts = LineItem('walnuts', 0, 10.00)

ValueError: value must be > 0

类中的特性能影响实例属性的寻找方式。特性都是类属性，但是特性惯例的其实是实例属性的存取。

如果实例和所属的类有同名数据属性，那么实例属性会覆盖（遮盖）类属性，至少通过那个实例读取属性时是这样。

obj.attr 这样的表达式不会从 obj 开始寻找 attr ，而是从 obj.\_\_class__ 开始，而且，仅当类中没有名为 attr 的特性时，Python 才会在 obj 实例中寻找。覆盖性描述符（overriding descriptor）也是如此。

In [47]:
# 两个属性：data、prop
class Class:
    data = 'the class data attr'
    @property
    def prop(self):
        return 'the prop value'

In [48]:
# 实例属性覆盖类的数据属性
obj = Class()
vars(obj)  # 返回 obj 的 __dict__ 属性

{}

In [49]:
obj.data  # 获取的是 Class.data 的值

'the class data attr'

In [50]:
obj.data = 'bar'  # 创建实例属性
vars(obj)  # 查看实例属性

{'data': 'bar'}

In [51]:
obj.data  # 获取的是 obj.data 的值

'bar'

In [52]:
Class.data  # 不影响 Class.data 的值

'the class data attr'

In [53]:
# 实例属性不会覆盖类特性
Class.prop  # 直接从 Class 中读取 prop 特性，获取的特性是对象本身，不会运行特性的读值方法

In [54]:
obj.prop  # 执行特性的读值方法

'the prop value'

In [55]:
obj.prop = 'foo'  # 设置实例属性失败

AttributeError: can't set attribute

In [56]:
obj.__dict__['prop'] = 'foo'  # 可以直接存入 obj.__dict__

In [57]:
vars(obj)  # 有 prop 属性

{'data': 'bar', 'prop': 'foo'}

In [58]:
obj.prop  # 执行特性的读值方法，特性没被实例属性覆盖

'the prop value'

In [59]:
Class.prop = 'baz'  # 覆盖特性，销毁特性对象（不再是特性）

In [60]:
obj.prop  # 获取的是实例属性

'foo'

In [61]:
# 新添的类特性覆盖现有的实例属性
obj.data  # 实例属性

'bar'

In [62]:
Class.data  # 类属性

'the class data attr'

In [63]:
Class.data = property(lambda self: 'the "data" prop value')  # 使用新特性覆盖

In [64]:
obj.data  # 被 Obj.data 特性覆盖

'the "data" prop value'

In [65]:
del Class.data  # 删除特性

In [66]:
obj.data  # 恢复实例属性

'bar'

各种 Python 代码单元（模块、函数、类、方法）都可以有文档字符串。

使用装饰器创建 property 对象时，读值方法（有 @property 装饰器的方法）的文档字符串作为一个整体，变成特性的文档。

In [67]:
class Foo:

    @property
    def bar(self):
        '''The bar attribute'''
        return self.__dict__['bar']

    @bar.setter
    def bar(self, value):
        self.__dict__['bar'] = value

In [68]:
f = Foo()
f.bar = 77
f.bar

77

In [69]:
Foo.bar.__doc__

'The bar attribute'

特性工厂函数

每次调用 quantity 工厂函数构建属性时，都要把 storage_name 参数设为独一无二的值。

In [70]:
# 特性工厂函数
def quantity(storage_name):  # storage_name 确定各个特性的数据存储在哪儿

    def qty_getter(instance):  # 取值
        return instance.__dict__[storage_name]  # 闭包，从 instance.__dict__ 中获取，跳过特性，防止无限递归

    def qty_setter(instance, value):  # 设值
        if value > 0:
            instance.__dict__[storage_name] = value  # 跳过特性，防止无限递归
        else:
            raise ValueError('value must be > 0')

    return property(qty_getter, qty_setter)  # 返回自定义的特性对象


class LineItem:
    weight = quantity('weight')  # 自定义特性
    price = quantity('price')  # 自定义特性

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight  # 特性已激活，不能设为零或负数
        self.price = price

    def subtotal(self):
        return self.weight * self.price  # 使用特性获取实例中的值

In [71]:
nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)
nutmeg.weight, nutmeg.price  # 通过特性读取，会覆盖同名实例属性

(8, 13.95)

In [72]:
sorted(vars(nutmeg).items())  # 查看真正用于存储值的实例属性

[('description', 'Moluccan nutmeg'), ('price', 13.95), ('weight', 8)]

对象的属性可以使用 del 语句删除：del my_object.an_attribute

定义特性时，可以使用 @my_propety.deleter 装饰器包装一个方法，负责删除特性管理的属性。

在不使用装饰器的经典调用句法中，fdel 参数用于设置删值函数。

如果不使用特性，还可以实现低层特殊的 \_\_delattr__ 方法处理删除属性的操作。

In [73]:
class BlackKnight:

    def __init__(self):
        self.members = ['an arm', 'another arm',
                        'a leg', 'another leg']
        self.phrases = ["'Tis but a scratch.",
                        "It's just a flesh wound.",
                        "I'm invincible!",
                        "All right, we'll call it a draw."]

    @property
    def member(self):
        print('next member is:')
        return self.members[0]

    @member.deleter
    def member(self):
        text = 'BLACK KNIGHT (loses {})\n-- {}'
        print(text.format(self.members.pop(0), self.phrases.pop(0)))

In [74]:
knight = BlackKnight()
knight.member

next member is:


'an arm'

In [75]:
del knight.member

BLACK KNIGHT (loses an arm)
-- 'Tis but a scratch.


In [76]:
del knight.member

BLACK KNIGHT (loses another arm)
-- It's just a flesh wound.


In [77]:
del knight.member

BLACK KNIGHT (loses a leg)
-- I'm invincible!


In [78]:
del knight.member

BLACK KNIGHT (loses another leg)
-- All right, we'll call it a draw.


影响属性处理方式的特殊属性
- \_\_class__
    - 对象所属类的引用，即 obj.\_\_class__ 与 type(obj) 的作用相同
- \_\_dict__
    - 一个映射，存储对象或类的可写属性
- \_\_slots__
    - 一个字符串组成的元组，类可以定义这个属性，限制实例能有哪些属性
    
处理属性的内置函数
- dir([object])
    - 列出对象的大多数属性；能审查有或没有 \_\_dict__ 属性的对象；不会列出类的几个特殊属性，例如 \_\_mro__、\_\_bases__、\_\_name__
- getattr(object, name[, default])
    - 从 object 对象中获取 name 字符串对应的属性
- hasattr(object, name)
    - 如果 object 对象中存在指定的属性，或者能以某种方式（例如继承）通过 object 对象获取指定的属性，返回 True
- setattr(object, name, value)
    - 把 object 对象指定属性的值设为 value，前提是 object 对象能接受那个值
- vars([object])
    - 返回 object 对象的 \_\_dict__ 属性；如果没有指定参数，那么 vars() 函数的作用与 locals() 函数一样：返回表示本地作用域的字典

使用点号或内置的 getattr、hasattr、setattr 函数存取属性都会触发下述列表中相应的特殊方法；直接通过实例的 \_\_dict__ 属性读写不会触发这些特殊方法。要假定特殊方法从类上获取，即便操作目标是实例。

处理属性的特殊方法（Class 类、obj 实例、attr 属性）
- \_\_delattr__(self, name)
    - 只要使用 del 语句删除属性，就会调用这个方法
- \_\_dir__(self)
    - 把对象传给 dir 函数时调用，列出属性
- \_\_getattr__(self, name)
    - 仅当获取指定的属性失败，搜索过 obj、Class、超类之后调用
- \_\_getattribute__(self, name)
    - 尝试获取指定的属性时总会调用这个方法，不过，寻找的属性是特殊属性或特殊方法时除外
    - 点号与 getattr、hasattr 内置函数会触发这个方法
    - 调用 \_\_getattribute__ 方法且抛出 AttributeError 异常时，才会调用 \_\_getattr__ 方法
- \_\_setattr__(self, name, value)
    - 尝试设置指定的属性时总会调用这个方法
    - 点号和 setattr 内置函数会出发这个方法

动态属性编程

使用 \_\_new__ 构造犯法把一个类转换成一个灵活的对象工厂函数

统一访问原则（Uniform Access Principle, UAP）